<center><img src="/files/images/DLI_Header.png" /></center>

# Star Bikes 营销文案生成器

在此 notebook 中，您将构建一个 AI 驱动的营销文案撰写器，来执行一些生成任务。您将学习如何编辑模型的**系统消息**（system message），以定义其响应。

## 学习目标

完成此 notebook 后，您将能够：
* 使用 LLaMA-2 执行各种**文本生成**任务。
* 使用**系统上下文**为 LLaMA-2 模型定义其首要（overarching）角色。

## 视频教程

执行以下单元以加载此 notebook 的视频教程。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/04-copy.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 工作流

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此 notebook 中，我们将使用以下函数与 LLM 交互。可以先大概看一眼，后面会详细介绍。

### 生成模型响应

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示词，包括可选的系统上下文和/或示例

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## Star Bikes 数据

In [ ]:
bikes = [
    {
        "model": "Galaxy Rider",
        "type": "Mountain",
        "features": {
            "frame": "Aluminum alloy",
            "gears": "21-speed Shimano",
            "brakes": "Hydraulic disc",
            "tires": "27.5-inch all-terrain",
            "suspension": "Full, adjustable",
            "color": "Matte black with green accents"
        },
        "usps": ["Lightweight frame", "Quick gear shift", "Durable tires"],
        "price": 799.95,
        "internal_id": "GR2321",
        "weight": "15.3 kg",
        "manufacturer_location": "Taiwan"
    },
    {
        "model": "Nebula Navigator",
        "type": "Hybrid",
        "features": {
            "frame": "Carbon fiber",
            "gears": "18-speed Nexus",
            "brakes": "Mechanical disc",
            "tires": "26-inch city slick",
            "suspension": "Front only",
            "color": "Glossy white"
        },
        "usps": ["Sleek design", "Efficient on both roads and trails", "Ultra-lightweight"],
        "price": 649.99,
        "internal_id": "NN4120",
        "weight": "13.5 kg",
        "manufacturer_location": "Germany"
    },
    {
        "model": "Cosmic Comet",
        "type": "Road",
        "features": {
            "frame": "Titanium",
            "gears": "24-speed Campagnolo",
            "brakes": "Rim brakes",
            "tires": "700C road",
            "suspension": "None",
            "color": "Metallic blue"
        },
        "usps": ["Super aerodynamic", "High-speed performance", "Professional-grade components"],
        "price": 1199.50,
        "internal_id": "CC5678",
        "weight": "11 kg",
        "manufacturer_location": "Italy"
    }
]

## 完整的 LLaMA-2 提示模板

在上一个 notebook 中，我们利用 LLaMA-2 **提示模板**来进行**少样本学习**，但也提到了我们使用的是略作修改的提示模板。具体来说，省略了被称为**系统消息**的用户消息、或者说是系统上下文或系统提示词（我们将交替使用这些术语）。下方是完整的 LLaMA-2 提示模板，包括了**系统上下文**。

```python
<s>[INST] <<SYS>>
{{ system_context }}
<</SYS>>

{{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>
```

**系统上下文**是用户/模型交互中用户侧的一部分，在 `<<SYS>>` 和 `<</SYS>>` 标签之间。**系统上下文**是将 AI 模型的响应导向特定框架或任务理解的线索。

关于**系统上下文**中应该写些什么内容，并没有短平快的规定。但我们可以将其视为对模型角色的设定，或是应作用到其所有响应的模式。

这是 LLaMA-2 聊天模型的默认**系统消息**，是指令微调期间用的：

>你是一个很有帮助、礼貌和诚实的助手。在安全的情况下，始终尽可能的提供帮助。你的回答不应包含任何有害、不道德、种族主义、性别主义、有毒、危险或违法的内容。请确保您的回答不带有社会偏见，并且是积极的。\
>如果一个问题没有任何意义，或与事实相悖，请进行解释而不是给出不正确的回复。如果你不知道问题的答案，请不要给出错误的信息。

## 设置系统上下文

下面的 `construct_prompt_with_context` 函数将帮助我们用 LLaMA-2 **提示模板**构建更新了**系统消息**的提示词。如果我们愿意，该函数还允许我们通过传递示例交互的二元组列表来执行**少样本学习**，跟上一节一样。

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## Star Bikes 营销文案生成器

让我们将 LLaMA-2 用作营销文案生成器。对于这个任务，我们将提供给模型关于 Star Bikes 自行车的相关规格。`bikes` 的定义可以在上方的 *Star Bikes 数据* 中查看。

我们先从一个简单的提示词开始。

In [ ]:
prompt = f"""
Write marketing copy for the following bicycle: {bikes[0]}
"""

print(generate(prompt))

---

看起来还不错，可以以此为基础继续迭代提示词来优化模型的响应。但假设我们希望模型充当营销文案撰写者，来编写多种格式的文案，那就可以为模型提供**系统上下文**让它知道自己的角色。

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

使用我们提供的 `construct_prompt_with_context` 函数可以创建一个按 LLaMA-2 提示模板要求的带有**系统上下文**的提示词。

In [ ]:
prompt_with_system_context = construct_prompt_with_context(prompt, system_context)
print(prompt_with_system_context)

---

使用带有**系统上下文**的提示词，现在看看模型会给我们什么样的响应。值得一提的是，我们的主 `prompt`（见上文）没有提供关于模型要做什么的指令。我们仅依赖**系统上下文**来指导模型的行为。

In [ ]:
print(generate(prompt_with_system_context))

---

一点也不赖，现在我们争取让**系统上下文**更**精确**，让模型只生成营销文案，而不要像刚刚的响应开头那样加一句：`Sure! Here's the marketing copy for the Galaxy Rider mountain bike:`。

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes. You only write marketing copy and never any \
leading comments or pieces of conversation.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

好像没起什么作用。让我们继续迭代。或许可以尝试告诉模型它是一台机器，这样就不会像人类那样对话了。

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

好多了！与所有的提示工程一样，开发有效的**系统提示**通常是一个迭代过程。

## 力求简洁

假设我们希望只得到大约 100 个词的响应，可以通过更新**系统上下文**来反映这一点。

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

非常好。现在的设置看起来很好用，让我们试试剩下的自行车数据：

In [ ]:
for bike in bikes[1:]:
    print(generate(construct_prompt_with_context(bike, system_context)))
    print("\n-----\n")

## 练习：生成营销电子邮件

根据您迄今为止所学的内容，创建一个可以给顾客撰写特定自行车营销邮件的提示词（利用**系统上下文**）。电子邮件应标明收件人的姓名。

如果遇到问题，请查看下方的参考答案。

### 您的代码

### 参考答案

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing emails in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
Recipient Name: Stella
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

## 关键概念回顾

此 notebook 介绍了以下关键概念：

* **系统消息**：指令微调模型提示模板的一部分，允许用户设置模型的角色或配置其总体行为。

## 可选的进阶练习

如果您想超出本课程的内容进阶一下，可以试试下面的额外开放式练习。

### 使用 7B 模型

在 notebook 顶部，按照下面的代码重启内核后，取消注释以使用 7B 模型，而不是 13B 模型。试试通过提示工程在使用小（更弱）模型的情况下获得满意的结果。

### 试验如何得到更好的提示词

我们通过对基本提示词进行迭代、编辑系统消息、提供示例（又叫“样本”）来帮助模型提升效果。与其说这是科学，不如说是一门艺术：看看您是否可以通过加强这 3 种改进来获得更好的效果。

### 创建电子邮件生成工作流

扩展上述练习中的工作，创建一个能给一组收件人分别生成邮件的工作流。除了参考收件人姓名之外，可以试试根据他们之前买过或感兴趣的自行车（无论之前是否发过邮件）来创建与这些细节更相关的电子邮件。

您还可以考虑用**少样本学习**以特定方式组织电子邮件。

重启内核
----




为下一个 notebook 释放 GPU 显存，请运行以下单元。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)